In [1]:
import pandas as pd

In [2]:
cd /Users/Ali/Desktop/Enron

/Users/Ali/Desktop/Enron


In [3]:
data= pd.read_csv('Enron.csv', sep=',')

In [4]:
data= data [['From', 'To']]

In [5]:
#data.head()

In [6]:
%%%time
data.fillna('{ }', inplace=True)
#fill NaN to perform operations

data = data.applymap(lambda x : x.split("{")[1])
data = data.applymap(lambda x : x.split("}")[0])
# split data on {} and to obtain clean email

data = data.applymap(str)
#just in case

CPU times: user 1.74 s, sys: 158 ms, total: 1.9 s
Wall time: 1.91 s


In [7]:
#data.head()

In [8]:
data = data[(data['From'].str.contains('enron')==True) & (data['To'].str.contains('enron')==True)]
#only concerned with enron employee emails

data = data.applymap(lambda x: x.replace("'", ''))
data = data.applymap(lambda x: x.replace('"', ''))
#remove all quotes from the DataFrame

In [9]:
data_to_list = data['To']
data_to_list = list(data_to_list)
data_to_list = [x.split(',') for x in data_to_list]
#convert to column to pandas series, to list to split and expand into a new Dataframe

data_to_frame = pd.DataFrame(data_to_list)

In [10]:
data_to_frame.head()

,0,1,2,3,4,5,6,7,8,9,...,880,881,882,883,884,885,886,887,888,889
0,tim.belden@enron.com,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,john.lavorato@enron.com,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,leah.arsdall@enron.com,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,randall.gay@enron.com,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,greg.piper@enron.com,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [11]:
data = data[['From']]
frames = [data,data_to_frame]
data = pd.concat(frames,axis=1,join_axes=[data.index])
#concat into new dataframe

In [12]:
data = data.applymap(lambda x : x if pd.isnull(x) else x.split('@')[0] )
#split on the @ to retrieve only sender names while skipping Null values

In [13]:
df = data.groupby('From').apply(lambda x : x.stack().value_counts())
#groupby sender and stack rows, count values in rows to return counts for every sender and recipient

In [14]:
df = pd.DataFrame(df)
df = df.reset_index()
df = df[(df['From']!=df['level_1'])]
#remove emails send to oneself
df = df.applymap(str)

In [15]:
data.head()

,From,0,1,2,3,4,5,6,7,8,...,880,881,882,883,884,885,886,887,888,889
0,phillip.allen,tim.belden,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,phillip.allen,john.lavorato,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,phillip.allen,leah.arsdall,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,phillip.allen,randall.gay,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,phillip.allen,greg.piper,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [16]:
import re

def find_digits(x):
    pattern = re.compile('\d')
    if pattern.search(x) == None:
        return x
    if pattern.search(x):
        return
    
#regex function to find any number in a string, we do not care about emails with numbers

In [17]:
df['From'] = df.From.map(find_digits)
df['level_1'] = df.level_1.map(find_digits)
df = df[(df['From'].isna()==False) & (df['level_1'].isna()==False)]
#map regex function to remove any rows with a number/digit identified

In [18]:
df.head()

,From,level_1,0
620,<d,linda.ward,1
621,<d,sigrid.macpherson,1
622,<d,gary.spraggins,1
623,<d,elizabeth.brown,1
624,<d,beverly.miller,1


In [19]:
df = df.applymap(lambda x : x.replace('.',' '))
df = df.applymap(lambda x : x.title())
#split on the period and title case names

In [20]:
df.columns = ['From','To','Count']
df['Count'] = df['Count'].map(int)
df = df[df['From'].str.contains('Enron')==False]
#remove emails from Enron accounts beacuse that is un-interesting

In [21]:
df.head()

,From,To,Count
620,<D,Linda Ward,1
621,<D,Sigrid Macpherson,1
622,<D,Gary Spraggins,1
623,<D,Elizabeth Brown,1
624,<D,Beverly Miller,1


In [22]:
positions = pd.read_html("enron_pos.html", 'r')
positions = positions[0]
positions = positions[[2,3]]
positions.columns = ['Name','Position']
of_interest = ['Vice President','Director','Trader','In House Lawyer', 'CEO']
positions = positions[positions['Position'].isin(of_interest)]
df_test = df.drop_duplicates('From')
check = positions.merge(df_test,left_on='Name', right_on='From', how='left')
check = check.replace({'Harpreet Arora':'Harry Arora','Michael Maggi':'Mike Maggi','Lawrence May':'Larry May','Matthew Motley':'Matt Motley','Jeffery Skilling':'Jeff Skilling','Carol St. Clair':'Carol Clair','Geoffery Storey':'Geoff Storey'})
key_players = list(check['Name'])
player_job = dict(zip(check['Name'],check['Position']))

#import job titles as html, create list of key players, i.e. ceo, directors, traders, VPs and Lawyer


ValueError: No tables found

In [ ]:
df_key_players = df[(df['From'].isin(key_players)) & (df['To'].isin(key_players))]
# create data set with only links between key players

In [ ]:
df_key_players['From'] = df_key_players['From'].apply(lambda x : (player_job[x] + ' - '+x))

In [ ]:
df_key_players['To'] = df_key_players['To'].apply(lambda x : (player_job[x] + ' - '+x))

In [ ]:
df_key_players.head()

In [ ]:
df_key_plays = df_key_players.pivot(index='From', columns = 'To',values = 'Count')
#pivot to create a matrix

In [ ]:
#df_key_plays.to_csv('df_key_players_3.csv')

In [ ]:
df_key_plays.head()

In [ ]:
df_key_players[df_key_players['From']=='Geoff Storey']

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
plt.matshow(df_key_plays.corr())

In [ ]:
def plot_corr(df_top_sends,size=10):
    '''Function plots a graphical correlation matrix for each pair of columns in the dataframe.

    Input:
        df: pandas DataFrame
        size: vertical and horizontal size of the plot'''

    corr = df_top_sends.corr()
    fig, ax = plt.subplots(figsize=(size, size))
    ax.matshow(corr)
    plt.xticks(range(len(corr.columns)), corr.columns);
    plt.yticks(range(len(corr.columns)), corr.columns);

In [ ]:
plot_corr(df_key_plays)

In [ ]:
from pandas.tools.plotting import scatter_matrix

In [ ]:
scatter_matrix(df_key_players, alpha=0.2, figsize=(6, 6), diagonal='kde')

In [ ]:
cd /Users/Ali/Desktop/Enron/D3

In [ ]:
positions = pd.read_html("enron_pos.html", 'r')
positions = positions[0]
positions = positions[[2,3]]
positions.columns = ['Name','Position']
of_interest = ['Vice President','Director','Trader','In House Lawyer', 'CEO']
positions = positions[positions['Position'].isin(of_interest)]
df_test = df.drop_duplicates('From')
check = positions.merge(df_test,left_on='Name', right_on='From', how='left')

In [ ]:
check.head()

In [ ]:
positions = pd.read_html("enron_pos.html", 'r')
positions = positions[0]
positions = positions[[2,3]]
positions.columns = ['Name','Position']
of_interest = ['Vice President','Director','Trader','In House Lawyer', 'CEO']
positions = positions[positions['Position'].isin(of_interest)]
df_test = df.drop_duplicates('From')
check = positions.merge(df_test,left_on='Name', right_on='From', how='left')
check = check.replace({'Harpreet Arora':'Harry Arora','Michael Maggi':'Mike Maggi','Lawrence May':'Larry May','Matthew Motley':'Matt Motley','Jeffery Skilling':'Jeff Skilling','Carol St. Clair':'Carol Clair','Geoffery Storey':'Geoff Storey'})
key_players = list(check['Name'])
player_job = dict(zip(check['Name'],check['Position']))

In [ ]:
player_job

In [ ]:
check = positions.merge(df_test,left_on='Name', right_on='From', how='left')
check = check[check['From'].isna()==False]
key_players = list(check['Name'])
adtl = list(['Harry Arora','Mike Maggi','Larry May','Matt Motley','Jeff Skilling','Carol Clair','Geoff Storey'])
key_players = key_players + adtl

#import job titles as html, create list of key players, i.e. ceo, directors, traders, VPs and Lawyer